![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [66]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [67]:
# Preprocess the data       
# Replace missing values 
cc_apps_nans_replaced = cc_apps.replace("?", np.NaN)

In [68]:
# Create a copy
cc_apps_imputed = cc_apps_nans_replaced.copy()

In [69]:
# Iterate over each column and impute the most frequent values for object dt and mean for numeric dt
for col in cc_apps_imputed.columns:
    if cc_apps_imputed[col].dtypes == 'object':
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(
                cc_apps_imputed[col].value_counts().index[0])
    else:
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(
                cc_apps_imputed[col].mean())

In [70]:
# one-hot encoding for missing values
cc_apps_encoded = pd.get_dummies(cc_apps_imputed, drop_first = True)

In [71]:
# Prepare the data for modeling
# Extracting last column for target variable
X = cc_apps_encoded.iloc[:, :-1].values
y = cc_apps_encoded.iloc[:, [-1]].values

In [72]:
# Split into train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [73]:
# Scale the data
# Instantiate StandardScaler to rescale X_train and X_test
scaler = StandardScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

In [74]:
# Train the model
# Instantiate LogisticRegression Classifierr
logreg = LogisticRegression()

In [75]:
# Fit logreg to train set
logreg.fit(rescaledX_train, y_train)

# Generating the predictions
y_train_pred = logreg.predict(rescaledX_train)

# Print the confusion matrix 
print(confusion_matrix(y_train, y_train_pred))


[[203   1]
 [  1 257]]


In [76]:
# Finding the best scoring model
# Defining Grid Search Parameters
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

In [77]:
# Create a dictionary 
param_grid = dict(tol = tol, max_iter = max_iter)

In [78]:
# Intantiate GridSearchCV
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv = 5)

In [79]:
# Fit grid_model to the data
grid_model_result = grid_model.fit(rescaledX_train, y_train)

In [80]:
# Summarize the results
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

Best: 0.818163 using {'max_iter': 100, 'tol': 0.01}


In [81]:
# Extract the best model and evaluate it
best_model = grid_model_result.best_estimator_
best_score = best_model.score(rescaledX_test, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

Accuracy of logistic regression classifier:  0.8289473684210527
